## Contextualized model

Let's check where the context really helps

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [1]:
%load_ext autoreload
%autoreload 2

from hatedetection import load_datasets

train_dataset, dev_dataset, test_dataset = load_datasets(add_body=True)


train_dataset = train_dataset.filter(lambda x: x["HATEFUL"] > 0)
dev_dataset = dev_dataset.filter(lambda x: x["HATEFUL"] > 0)
test_dataset = test_dataset.filter(lambda x: x["HATEFUL"] > 0)

In [2]:
import torch
from hatedetection.model import BertForSequenceMultiClassification
from hatedetection import extended_hate_categories
from transformers import AutoModelForSequenceClassification, AutoTokenizer

no_context_model_name = "../models/bert-non-contextualized-hate-category-es/"
context_model_name = "../models/bert-contextualized-hate-category-es/"
full_context_model_name = "../models/bert-title+body-hate-category-es/"


no_context_model = BertForSequenceMultiClassification.from_pretrained(no_context_model_name, num_labels=len(extended_hate_categories))
context_model = BertForSequenceMultiClassification.from_pretrained(context_model_name, num_labels=len(extended_hate_categories))
full_context_model = BertForSequenceMultiClassification.from_pretrained(full_context_model_name, num_labels=len(extended_hate_categories))

context_model.eval()
no_context_model.eval()
full_context_model.eval()


device = "cuda" if torch.cuda.is_available() else "cpu"

context_model = context_model.to(device)
no_context_model = no_context_model.to(device)
full_context_model = full_context_model.to(device)
# Tienen mismo tokenizer así que todo bien

no_context_tokenizer = AutoTokenizer.from_pretrained(no_context_model_name)
context_tokenizer = AutoTokenizer.from_pretrained(context_model_name)
full_context_tokenizer = AutoTokenizer.from_pretrained(full_context_model_name)

no_context_tokenizer.model_max_length = 128
context_tokenizer.model_max_length = 256

In [3]:
from hatedetection.training import tokenize

batch_size = 32
eval_batch_size = 32

dataset = test_dataset


no_context_dataset = dataset.map(
    lambda x: tokenize(no_context_tokenizer, x, context='none'), batched=True, batch_size=eval_batch_size)
context_dataset = dataset.map(
    lambda x: tokenize(context_tokenizer, x, context='title'), batched=True, batch_size=eval_batch_size)
full_context_dataset = dataset.map(
    lambda x: tokenize(full_context_tokenizer, x, context='title+body'), batched=True, batch_size=8)


In [4]:
full_context_tokenizer.decode(full_context_dataset[0]["input_ids"])

'[CLS] [ USER ] Faltó que se enganchó al hijo de sofovich para seguir adelante [ EMOJI ] cara con mano sobre la boca [ EMOJI ] [SEP] Mora Godoy cierra su escuela de tango y remata su vestuario para " poder seguir adelante " - Puede marcar con justeza la fecha de inicio de la crisis : hace 10 meses que no brinda clases. En cambio, no logra avizorar el día que concluirá : " No sabemos cuándo empezamos, cuando volvemos a trabajar. ¿ Ideas? ¡ ( Tenemos ) miles! Podemos hacer un show gratis al aire libre ". Es por eso que, ante la falta de un horizonte, Mora Godoy decidió cerrar su escuela de danzas y rematar gran parte del vestuario para poder continuar adelante. " Hace 20 años que estoy acá, pero llevo diez meses sin trabajar - le explicó la artista a Guillermo Andino en el programa Informados de todo, por América -. Este es mi lugar de ensayo, de creación. Y es muy duro. Para poder seguir adelante me tengo que desprender del lugar y rematar gran parte del vestuario. Que muchos bailarines

In [5]:

def format_dataset(dataset):
    def get_category_labels(examples):
        return {'labels': torch.Tensor([examples[cat] for cat in extended_hate_categories])}
    dataset = dataset.map(get_category_labels)
    dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
    return dataset

no_context_dataset = format_dataset(no_context_dataset)
context_dataset = format_dataset(context_dataset)
full_context_dataset = format_dataset(full_context_dataset)


In [6]:
full_context_dataset["input_ids"].shape, context_dataset["input_ids"].shape, no_context_dataset["input_ids"].shape

(torch.Size([1797, 512]), torch.Size([1797, 256]), torch.Size([1797, 128]))

Sanity check

In [7]:
no_context_tokenizer.decode(no_context_dataset[0]["input_ids"])

'[CLS] [USER] Faltó que se enganchó al hijo de sofovich para seguir adelante [EMOJI] cara con mano sobre la boca [EMOJI] [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [8]:
context_tokenizer.decode(context_dataset[0]["input_ids"])

'[CLS] [ USER ] Faltó que se enganchó al hijo de sofovich para seguir adelante [ EMOJI ] cara con mano sobre la boca [ EMOJI ] [SEP] Mora Godoy cierra su escuela de tango y remata su vestuario para " poder seguir adelante " [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P

Bien, primero contexto

Lo cargamos sólo para evaluar 🤗

In [9]:
from hatedetection.metrics import compute_category_metrics
from transformers import Trainer, TrainingArguments

eval_batch_size = 32

training_args = TrainingArguments(
    output_dir=".",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=eval_batch_size,
)



no_context_trainer = Trainer(
    model=no_context_model,
    args=training_args,
    compute_metrics=compute_category_metrics,
)

context_trainer = Trainer(
    model=context_model,
    args=training_args,
    compute_metrics=compute_category_metrics,
)


full_context_trainer = Trainer(
    model=full_context_model,
    args=TrainingArguments(
        output_dir=".",
        per_device_eval_batch_size=32,
    ),
    compute_metrics=compute_category_metrics,
)


## Resultados

In [10]:
import pandas as pd
pd.options.display.max_columns = 40
pd.set_option('display.float_format', lambda x: '%.5f' % x)


df_full_context_results = pd.DataFrame([full_context_trainer.evaluate(full_context_dataset)])
df_context_results = pd.DataFrame([context_trainer.evaluate(context_dataset)])
df_no_context_results = pd.DataFrame([no_context_trainer.evaluate(no_context_dataset)])


In [14]:
df_results = pd.concat([
    df_no_context_results, 
    df_context_results, 
    df_full_context_results
])
df_results["index"] = ["No context", "Title", "Title and Body"]
df_results.set_index("index")

,eval_loss,eval_calls_f1,eval_women_f1,eval_lgbti_f1,eval_racism_f1,eval_class_f1,eval_politics_f1,eval_disabled_f1,eval_appearance_f1,eval_criminal_f1,eval_mean_f1,eval_mean_precision,eval_mean_recall,eval_runtime,eval_samples_per_second,init_mem_cpu_alloc_delta,init_mem_gpu_alloc_delta,init_mem_cpu_peaked_delta,init_mem_gpu_peaked_delta,eval_mem_cpu_alloc_delta,eval_mem_gpu_alloc_delta,eval_mem_cpu_peaked_delta,eval_mem_gpu_peaked_delta
index,,,,,,,,,,,,,,,,,,,,,,,
No context,0.21134,0.78119,0.65497,0.57407,0.86674,0.59701,0.71193,0.78367,0.84522,0.74487,0.72885,0.78101,0.68699,6.36710,282.23200,52465,0,18258,0,246892,0,299056,138739200
Title,0.15317,0.79259,0.66667,0.81644,0.94227,0.72325,0.74157,0.72581,0.89037,0.90733,0.80070,0.84489,0.76362,12.89800,139.32400,43213,0,18258,0,252053,0,299888,377929216
Title and Body,0.16149,0.75915,0.70472,0.81319,0.93731,0.74820,0.75147,0.77206,0.88799,0.89510,0.80769,0.84901,0.77449,29.21040,61.51900,43460,0,18258,0,418372,0,284772,1158299136
